# Modern monetary/Keynesian approach

## sticky monetary approach

multistep: mutistpe forecast  
m: quarterly data

### dependent variable

- "DEXJPUS","fred"

### independent variable

- "MANMM101USM189S","fred"
- "MANMM101JPM189S","fred"
- "GDP","fred"
- "JPNNGDP","fred"
- "MKTGDPJPA646NWDB","fred"
- "USD12MD156N","fred"
- "JPY12MD156N","fred"
- "PPIACO","fred"
- "PITGCG01JPM661N","fred"

In [1]:
%matplotlib inline
import datetime
import pandas_datareader.data as web
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.metrics import mean_squared_error
from statsmodels.tsa.api import VECM
start="1986/1/1";end='2019/12/31'
start1="1985/1/1";end1='2019/12/31'
from dateutil.relativedelta import relativedelta
from scipy.stats import t
from scipy.stats import norm
def tcdf(x,n): #t-distribution
    if x>=0:
        y=(1-t.cdf(x,n))
    else:
        y=t.cdf(x,n)
    return y
def ncdf(x): #t-distribution
    if x>=0:
        y=(1-norm.cdf(x))
    else:
        y=norm.cdf(x)
    return y

In [2]:
e =web.DataReader("DEXJPUS","fred",start,end)
m1 =web.DataReader("MANMM101USM189S","fred",start,end)/1000000000#w
m2 =web.DataReader("MANMM101JPM189S","fred",start,end)/1000000000#w
y1 =web.DataReader("GDP","fred",start,end)# norminal q
y2 =web.DataReader("JPNNGDP","fred",start,end)#nominal q
y3=web.DataReader("MKTGDPJPA646NWDB","fred",start,end)#q
r1 =web.DataReader("USD12MD156N","fred",start,end)#q
r2 =web.DataReader("JPY12MD156N","fred",start,end)#q
p3 =web.DataReader("PPIACO","fred",start1,end)#m
p4 =web.DataReader("PITGCG01JPM661N","fred",start1,end)#m
#y1 =web.DataReader("GDPC1","fred",start,end)# real q
#y2 =web.DataReader("JPNRGDPEXP","fred",start,end)# real

In [3]:
FRQ=['QS-JAN']
interval=3
for i in range(len(FRQ)):
    ee=e.copy().resample(FRQ[i]).last()
    ee.columns=['x']
    mm1=m1.copy().resample(FRQ[i]).last()
    mm1.columns=['x']
    mm2=m2.copy().resample(FRQ[i]).last()
    mm2.columns=['x']
    yy1=y1.copy().resample(FRQ[i]).last()
    yy1.columns=['x']
    yy2=y2.copy().resample(FRQ[i]).last()
    yy2.columns=['x']
    rr1=r1.copy().resample(FRQ[i]).last()
    rr1.columns=['x']
    rr2=r2.copy().resample(FRQ[i]).last()
    rr2.columns=['x']
    pp1=(p3.copy().pct_change().rolling(12).mean().dropna()).resample(FRQ[i]).last()
    pp1.columns=['x']
    pp2=(p4.copy().pct_change().rolling(12).mean().dropna()).resample(FRQ[i]).last()
    pp2.columns=['x']
    tsd=pd.concat([ee,mm1,mm2,yy1,yy2,rr1,rr2,pp1,pp2,rr1/100-pp1,rr2/100-pp2],axis=1)
    tsd.columns=['ee','m1','m2','y1','y2','r1','r2','p1','p2','rp1','rp2']
    #tsd=tsd.fillna(0)
    idx=e.index[-1]-relativedelta(months=2)
    tsd=tsd.copy().loc[:idx]
    print(i,FRQ[i],tsd.tail(5))
    if i==0:
        tsd1=tsd
    elif i==1:
        tsd2=tsd
    else:
        tsd3=tsd

0 QS-JAN                 ee      m1        m2         y1        y2       r1       r2  \
DATE                                                                          
2018-10-01  109.70  3746.5  772898.2  20897.804  546737.7  3.00544  0.10633   
2019-01-01  110.68  3730.0  783525.1  21098.827  552687.8  2.71063  0.09250   
2019-04-01  107.84  3832.9  793923.2  21340.267  555954.0  2.17813  0.04567   
2019-07-01  108.11  3903.3  804952.1  21542.540  558237.1  2.03213  0.02550   
2019-10-01  108.67  3978.4  817294.3  21729.124  549920.9  1.99625  0.10483   

                  p1        p2       rp1       rp2  
DATE                                                
2018-10-01  0.001996 -0.001016  0.028058  0.002079  
2019-01-01  0.000651 -0.000329  0.026455  0.001254  
2019-04-01 -0.001588 -0.001102  0.023369  0.001559  
2019-07-01 -0.002134 -0.001708  0.022456  0.001963  
2019-10-01 -0.000736 -0.000939  0.020698  0.001987  


In [4]:
y4=y3.resample('QS').asfreq()
y4=y4.interpolate()/1000000000
y5=pd.concat([tsd1.ee,y4,y2],axis=1)
y5.columns=['e','y4','y2']
y5.iloc[:,1]=y5.y4*y5.e
y6=(y5.y4.rolling(window=8).mean()).shift(-6)
y6=y6.bfill()
l=len(y5.loc['1994':])
y6.iloc[l:]=y5.iloc[l:,2].values
tsd1=pd.concat([ee,mm1,mm2,yy1,y6,rr1,rr2,pp1,pp2,rr1/100-pp1,rr2/100-pp2],axis=1)
tsd1.columns=['ee','m1','m2','y1','y2','r1','r2','p1','p2','rp1','rp2']
tsd=tsd1

In [5]:
def backtest_ols_dp(tsd,tw,pt):#tw time window pt:predicting time
    pred=[]
    real0=[]
    real9=[]
    tsd0=np.log(tsd.iloc[:,:6]).copy()
    for i in range(tw+1,len(tsd)+1-1):
        yy=tsd0.iloc[i-tw:i:1,0]
        y0=tsd0.iloc[i-tw-1:i:1,0].diff().dropna()
        x0=(tsd0.iloc[i-tw-1:i:1,1]-tsd0.iloc[i-tw-1:i:1,2]).diff().dropna()
        x00=(tsd0.iloc[i-tw-1:i:1,3]-tsd0.iloc[i-tw-1:i:1,4]).diff().dropna()
        x000=((1+tsd.iloc[i-tw-1:i:1,6]/100)/(1+tsd.iloc[i-tw-1:i:1,5]/100)).diff().dropna()
        x0000=((1+tsd.iloc[i-tw-1:i:1,8])/(1+tsd.iloc[i-tw-1:i:1,7])).diff().dropna()
        x0=pd.concat([x0,x00,x000,x0000],axis=1)
        x0=sm.add_constant(x0.values)
        res= sm.OLS(y0, x0).fit()
        real0.append(np.exp(tsd0.iloc[i-1:i,0][-1]))
        if i<len(tsd)+1-pt:
            x9=(tsd0.iloc[i-1:i+pt:1,1]-tsd0.iloc[i-1:i+pt:1,2]).diff().dropna()
            x99=(tsd0.iloc[i-1:i+pt:1,3]-tsd0.iloc[i-1:i+pt:1,4]).diff().dropna()
            x999=((1+tsd.iloc[i-1:i+pt:1,6]/100)/(1+tsd.iloc[i-1:i+pt:1,5]/100)).diff().dropna()
            x9999=((1+tsd.iloc[i-1:i+pt:1,8])/(1+tsd.iloc[i-1:i+pt:1,7])).diff().dropna()
            x9=pd.concat([x9,x99,x999,x9999],axis=1)
            if pt==1:
                x9=np.insert(x9.values,0,1)
            else:
                x9=sm.add_constant(x9.values)
            pred0=res.predict(x9)[-1:]
            pred.append(np.exp(pred0+yy.iloc[-1]))
            real9.append(np.exp(tsd0.iloc[i+pt-1:i+pt:1,0].values))
    return real0,real9,pred

def performance_eval(real0,real9,pred):
    p0=pd.DataFrame(real0)
    p9=pd.DataFrame(real9)
    pp=pd.DataFrame(pred)
    p0_=p0[:len(p9)]
    pp_=pp[:len(p9)]
    wrs=p0_[(p9<p0_)&(pp_<p0_)].count()
    wrl=p0_[(p9>p0_)&(pp_>p0_)].count()
    wr=float((wrs+wrl)/p0_.count())
    #wrp=tcdf(float((wr-0.5)/np.sqrt(0.25/p0_.count())),p0_.count())[0]
    wrp=ncdf(float((wr-0.5)/np.sqrt(0.25/p0_.count())))
    d_x=(pp_-p9)**2
    d_rw=(p9-p0_)**2
    d=d_x-d_rw
    d_=float(d.mean())
    #dm=tcdf(float(d_/np.sqrt(((d-d_)**2).sum()/len(d)**2)),p0_.count())[0]
    dm=ncdf(float(d_/np.sqrt(((d-d_)**2).sum()/len(d)**2)))
    rmse=float(d_x.mean()/d_rw.mean())
    return rmse,dm,wr,wrp


def performance_report(real0,real9,pred):
    ab,dm,wr,wrp=performance_eval(real0,real9,pred)
    print('{0:20s}{1:5d} mse_r {2:7.4f} mse_p {3:7.2f} wr {4:7.4f} wrp {5:7.2f} len {6:4d}'
          .format(model,isp,ab,dm,wr,wrp,len(real9)))    

In [6]:
model='OLS'
oosp=1
pstart=datetime.date(1991,1,1)
pend=datetime.date(1994,12,31)        
nn=len(tsd.loc[pstart:pend])
for isp in range(oosp*10,oosp*50,3):#10,pt):
    ps=pstart-relativedelta(months=isp*interval-1)
    tsdd=tsd.loc[ps:pend]
    if (len(tsdd)-isp==nn) or (len(tsdd)-isp==nn-1):
        real0,real9,pred=backtest_ols_dp(tsdd,isp,oosp)
        performance_report(real0,real9,pred)

OLS                    10 mse_r  4.2619 mse_p    0.13 wr  0.7143 wrp    0.05 len   14
OLS                    13 mse_r  2.8304 mse_p    0.12 wr  0.5000 wrp    0.50 len   14
OLS                    16 mse_r  2.4353 mse_p    0.15 wr  0.5000 wrp    0.50 len   14
OLS                    19 mse_r  2.6431 mse_p    0.17 wr  0.5000 wrp    0.50 len   14


In [7]:
model='OLS'
oosp=1
pstart=datetime.date(1995,1,1)
pend=datetime.date(2007,12,31)        
nn=len(tsd.loc[pstart:pend])
for isp in range(oosp*10,oosp*50,3):#10,pt):
    ps=pstart-relativedelta(months=isp*interval-1)
    tsdd=tsd.loc[ps:pend]
    if (len(tsdd)-isp==nn) or (len(tsdd)-isp==nn-1):
        real0,real9,pred=backtest_ols_dp(tsdd,isp,oosp)
        performance_report(real0,real9,pred)

OLS                    10 mse_r  2.3937 mse_p    0.01 wr  0.6000 wrp    0.08 len   50
OLS                    13 mse_r  1.9807 mse_p    0.03 wr  0.5800 wrp    0.13 len   50
OLS                    16 mse_r  1.2830 mse_p    0.16 wr  0.5800 wrp    0.13 len   50
OLS                    19 mse_r  0.9176 mse_p    0.37 wr  0.6200 wrp    0.04 len   50
OLS                    22 mse_r  0.8715 mse_p    0.28 wr  0.6000 wrp    0.08 len   50
OLS                    25 mse_r  0.9065 mse_p    0.33 wr  0.6400 wrp    0.02 len   50
OLS                    28 mse_r  0.9154 mse_p    0.35 wr  0.6400 wrp    0.02 len   50
OLS                    31 mse_r  0.9072 mse_p    0.32 wr  0.7200 wrp    0.00 len   50
OLS                    34 mse_r  0.9104 mse_p    0.30 wr  0.7000 wrp    0.00 len   50
OLS                    37 mse_r  0.8975 mse_p    0.28 wr  0.6800 wrp    0.01 len   50


In [8]:
model='OLS'
oosp=1
pstart=datetime.date(2008,1,1)
pend=datetime.date(2020,12,31)        
nn=len(tsd.loc[pstart:pend])
for isp in range(oosp*10,oosp*50,3):#10,pt):
    ps=pstart-relativedelta(months=isp*interval-1)
    tsdd=tsd.loc[ps:pend]
    if (len(tsdd)-isp==nn) or (len(tsdd)-isp==nn-1):
        real0,real9,pred=backtest_ols_dp(tsdd,isp,oosp)
        performance_report(real0,real9,pred)

OLS                    10 mse_r  1.6294 mse_p    0.03 wr  0.6087 wrp    0.07 len   46
OLS                    13 mse_r  1.5877 mse_p    0.04 wr  0.6087 wrp    0.07 len   46
OLS                    16 mse_r  1.2404 mse_p    0.15 wr  0.6087 wrp    0.07 len   46
OLS                    19 mse_r  1.1385 mse_p    0.29 wr  0.6304 wrp    0.04 len   46
OLS                    22 mse_r  1.0700 mse_p    0.37 wr  0.6522 wrp    0.02 len   46
OLS                    25 mse_r  1.1424 mse_p    0.28 wr  0.6304 wrp    0.04 len   46
OLS                    28 mse_r  1.2920 mse_p    0.08 wr  0.6304 wrp    0.04 len   46
OLS                    31 mse_r  1.3232 mse_p    0.05 wr  0.6087 wrp    0.07 len   46
OLS                    34 mse_r  1.2713 mse_p    0.08 wr  0.6087 wrp    0.07 len   46
OLS                    37 mse_r  1.3725 mse_p    0.04 wr  0.5870 wrp    0.12 len   46
OLS                    40 mse_r  1.3013 mse_p    0.05 wr  0.5870 wrp    0.12 len   46
OLS                    43 mse_r  1.2818 mse_p    0.07 

In [9]:
model='OLS'
oosp=4
pstart=datetime.date(1991,1,1)
pend=datetime.date(1994,12,31)       
nn=len(tsd.loc[pstart:pend])
for isp in range(oosp*3,oosp*50,3):#10,pt):
    ps=pstart-relativedelta(months=isp*interval-1)
    tsdd=tsd.loc[ps:pend]
    if (len(tsdd)-isp==nn) or (len(tsdd)-isp==nn-1):
        real0,real9,pred=backtest_ols_dp(tsdd,isp,oosp)
        performance_report(real0,real9,pred)

OLS                    12 mse_r  1.1832 mse_p    0.28 wr  0.8182 wrp    0.02 len   11
OLS                    15 mse_r  1.0331 mse_p    0.44 wr  0.6364 wrp    0.18 len   11
OLS                    18 mse_r  1.0126 mse_p    0.48 wr  0.6364 wrp    0.18 len   11
OLS                    21 mse_r  0.7143 mse_p    0.08 wr  0.9091 wrp    0.00 len   11


In [10]:
model='OLS'
oosp=4
pstart=datetime.date(1995,1,1)
pend=datetime.date(2007,12,31)       
nn=len(tsd.loc[pstart:pend])
for isp in range(oosp*3,oosp*50,6):#10,pt):
    ps=pstart-relativedelta(months=isp*interval-1)
    tsdd=tsd.loc[ps:pend]
    if (len(tsdd)-isp==nn) or (len(tsdd)-isp==nn-1):
        real0,real9,pred=backtest_ols_dp(tsdd,isp,oosp)
        performance_report(real0,real9,pred)

OLS                    12 mse_r  2.4315 mse_p    0.05 wr  0.5319 wrp    0.33 len   47
OLS                    18 mse_r  0.9411 mse_p    0.32 wr  0.6170 wrp    0.05 len   47
OLS                    24 mse_r  0.9896 mse_p    0.47 wr  0.5957 wrp    0.09 len   47
OLS                    30 mse_r  1.0227 mse_p    0.42 wr  0.5745 wrp    0.15 len   47
OLS                    36 mse_r  0.8799 mse_p    0.11 wr  0.5745 wrp    0.15 len   47


In [11]:
model='OLS'
oosp=4
pstart=datetime.date(2008,1,1)
pend=datetime.date(2020,12,31)       
nn=len(tsd.loc[pstart:pend])
for isp in range(oosp*3,oosp*50,6):#10,pt):
    ps=pstart-relativedelta(months=isp*interval-1)
    tsdd=tsd.loc[ps:pend]
    if (len(tsdd)-isp==nn) or (len(tsdd)-isp==nn-1):
        real0,real9,pred=backtest_ols_dp(tsdd,isp,oosp)
        performance_report(real0,real9,pred)

OLS                    12 mse_r  1.2255 mse_p    0.11 wr  0.5349 wrp    0.32 len   43
OLS                    18 mse_r  0.9637 mse_p    0.34 wr  0.5349 wrp    0.32 len   43
OLS                    24 mse_r  1.0122 mse_p    0.45 wr  0.6279 wrp    0.05 len   43
OLS                    30 mse_r  1.0488 mse_p    0.32 wr  0.5349 wrp    0.32 len   43
OLS                    36 mse_r  1.0619 mse_p    0.24 wr  0.5349 wrp    0.32 len   43
OLS                    42 mse_r  1.0336 mse_p    0.33 wr  0.5349 wrp    0.32 len   43
OLS                    48 mse_r  0.9926 mse_p    0.46 wr  0.5581 wrp    0.22 len   43
OLS                    54 mse_r  0.9726 mse_p    0.35 wr  0.5814 wrp    0.14 len   43
OLS                    60 mse_r  0.9536 mse_p    0.25 wr  0.5581 wrp    0.22 len   43
OLS                    66 mse_r  0.9051 mse_p    0.09 wr  0.6279 wrp    0.05 len   43
OLS                    72 mse_r  0.8943 mse_p    0.06 wr  0.5581 wrp    0.22 len   43
OLS                    78 mse_r  0.9152 mse_p    0.11 

In [12]:
model='OLS'
oosp=20
pstart=datetime.date(1995,1,1)
pend=datetime.date(2007,12,31)   
nn=len(tsd.loc[pstart:pend])        
for isp in range(12,oosp*10,2):#10,pt):
    ps=pstart-relativedelta(months=isp*interval-1)
    tsdd=tsd.loc[ps:pend]
    if (len(tsdd)-isp==nn) or (len(tsdd)-isp==nn-1):
        real0,real9,pred=backtest_ols_dp(tsdd,isp,oosp)
        performance_report(real0,real9,pred)

OLS                    12 mse_r  2.1788 mse_p    0.00 wr  0.5161 wrp    0.43 len   31
OLS                    14 mse_r  1.6340 mse_p    0.00 wr  0.5161 wrp    0.43 len   31
OLS                    16 mse_r  1.2869 mse_p    0.06 wr  0.4839 wrp    0.43 len   31
OLS                    18 mse_r  1.0979 mse_p    0.28 wr  0.5484 wrp    0.30 len   31
OLS                    20 mse_r  1.2191 mse_p    0.08 wr  0.4516 wrp    0.30 len   31
OLS                    22 mse_r  1.2036 mse_p    0.11 wr  0.4516 wrp    0.30 len   31
OLS                    24 mse_r  1.2188 mse_p    0.09 wr  0.4516 wrp    0.30 len   31
OLS                    26 mse_r  1.1054 mse_p    0.24 wr  0.4839 wrp    0.43 len   31
OLS                    28 mse_r  0.9510 mse_p    0.36 wr  0.6129 wrp    0.10 len   31
OLS                    30 mse_r  0.8844 mse_p    0.19 wr  0.6129 wrp    0.10 len   31
OLS                    32 mse_r  0.8490 mse_p    0.11 wr  0.5806 wrp    0.18 len   31
OLS                    34 mse_r  0.8670 mse_p    0.14 

In [13]:
model='OLS'
oosp=20
pstart=datetime.date(2008,1,1)
pend=datetime.date(2020,12,31)   
nn=len(tsd.loc[pstart:pend])        
for isp in range(15,oosp*10,12):#10,pt):
    ps=pstart-relativedelta(months=isp*interval-1)
    tsdd=tsd.loc[ps:pend]
    if (len(tsdd)-isp==nn) or (len(tsdd)-isp==nn-1):
        real0,real9,pred=backtest_ols_dp(tsdd,isp,oosp)
        performance_report(real0,real9,pred)

OLS                    15 mse_r  0.8848 mse_p    0.00 wr  0.7407 wrp    0.01 len   27
OLS                    27 mse_r  0.9205 mse_p    0.00 wr  0.6667 wrp    0.04 len   27
OLS                    39 mse_r  0.9504 mse_p    0.01 wr  0.7037 wrp    0.02 len   27
OLS                    51 mse_r  0.9405 mse_p    0.02 wr  0.7037 wrp    0.02 len   27
OLS                    63 mse_r  0.9288 mse_p    0.01 wr  0.6667 wrp    0.04 len   27
OLS                    75 mse_r  0.9561 mse_p    0.04 wr  0.6296 wrp    0.09 len   27
OLS                    87 mse_r  0.9712 mse_p    0.09 wr  0.6296 wrp    0.09 len   27


In [14]:
def backtest_VECM(tsd,tw,pt,det):#tw time window pt:predicting time
    pred=[]
    real0=[]
    real9=[]
    tsd0=np.log(tsd.iloc[:,:6]).copy()    
    for i in range(tw,len(tsd)+1):
        y0=(tsd0.iloc[i-tw:i:1,0])
        x=tsd0.iloc[i-tw:i:1,1]-tsd0.iloc[i-tw:i:1,2]
        xx=tsd0.iloc[i-tw:i:1,3]-tsd0.iloc[i-tw:i:1,4]
        xxx=(1+tsd.iloc[i-tw:i:1,6]/100)/(1+tsd.iloc[i-tw:i:1,5]/100)
        xxxx=(1+tsd.iloc[i-tw:i:1,8])/(1+tsd.iloc[i-tw:i:1,7])
        tsd00=pd.concat([y0,x,xx,xxx],axis=1)
        model = VECM(tsd00,k_ar_diff=0,coint_rank=1,deterministic=det)
        results = model.fit() 
        real0.append(np.exp(y0.iloc[-1]))
        pp=np.exp(results.predict(pt)[-1][0])
        pred.append(pp)
        if i<len(tsd)+1-pt:
            real9.append(tsd.iloc[i+pt-1,0])
    return real0,real9,pred

In [15]:
model='VECM'
oosp=1
pstart=datetime.date(1991,1,1)
pend=datetime.date(1994,12,31)   
nn=len(tsd.loc[pstart:pend])    
for f in range(len(FRQ)):
    for isp in range(oosp*10,oosp*200,oosp*1):#10,pt):
        ps=pstart-relativedelta(months=isp*interval-1)
        tsdd=tsd.loc[ps:pend]
        if (len(tsdd)-isp==nn) or (len(tsdd)-isp==nn-1):
            real0,real9,pred=backtest_VECM(tsdd,isp,oosp,'li')
            performance_report(real0,real9,pred)

VECM                   10 mse_r  1.2881 mse_p    0.16 wr  0.4667 wrp    0.40 len   15
VECM                   11 mse_r  1.8511 mse_p    0.03 wr  0.5333 wrp    0.40 len   15
VECM                   12 mse_r  1.1437 mse_p    0.32 wr  0.5333 wrp    0.40 len   15
VECM                   13 mse_r  0.9418 mse_p    0.40 wr  0.5333 wrp    0.40 len   15
VECM                   14 mse_r  0.8884 mse_p    0.23 wr  0.6000 wrp    0.22 len   15
VECM                   15 mse_r  1.1208 mse_p    0.32 wr  0.4000 wrp    0.22 len   15
VECM                   16 mse_r  1.1909 mse_p    0.29 wr  0.5333 wrp    0.40 len   15
VECM                   17 mse_r  0.9452 mse_p    0.43 wr  0.4667 wrp    0.40 len   15
VECM                   18 mse_r  0.9815 mse_p    0.47 wr  0.6000 wrp    0.22 len   15
VECM                   19 mse_r  1.2483 mse_p    0.19 wr  0.4000 wrp    0.22 len   15
VECM                   20 mse_r  1.1634 mse_p    0.28 wr  0.4000 wrp    0.22 len   15
VECM                   21 mse_r  1.0617 mse_p    0.40 

In [16]:
model='VECM'
oosp=1
pstart=datetime.date(1995,1,1)
pend=datetime.date(2007,12,31)   
nn=len(tsd.loc[pstart:pend])    
for isp in range(oosp*18,oosp*200,oosp*2):#10,pt):
    ps=pstart-relativedelta(months=isp*interval-1)
    tsdd=tsd.loc[ps:pend]
    if (len(tsdd)-isp==nn) or (len(tsdd)-isp==nn-1):
        real0,real9,pred=backtest_VECM(tsdd,isp,oosp,'ci')
        performance_report(real0,real9,pred)

VECM                   18 mse_r  1.0629 mse_p    0.13 wr  0.6275 wrp    0.03 len   51
VECM                   20 mse_r  1.0125 mse_p    0.38 wr  0.6275 wrp    0.03 len   51
VECM                   22 mse_r  1.0527 mse_p    0.21 wr  0.6275 wrp    0.03 len   51
VECM                   24 mse_r  1.0981 mse_p    0.07 wr  0.5686 wrp    0.16 len   51
VECM                   26 mse_r  1.0764 mse_p    0.11 wr  0.5098 wrp    0.44 len   51
VECM                   28 mse_r  1.1044 mse_p    0.05 wr  0.4902 wrp    0.44 len   51
VECM                   30 mse_r  1.1108 mse_p    0.04 wr  0.4706 wrp    0.34 len   51
VECM                   32 mse_r  1.1520 mse_p    0.02 wr  0.4314 wrp    0.16 len   51
VECM                   34 mse_r  1.1401 mse_p    0.01 wr  0.3529 wrp    0.02 len   51
VECM                   36 mse_r  1.0713 mse_p    0.07 wr  0.4314 wrp    0.16 len   51


In [17]:
model='VECM'
oosp=1
pstart=datetime.date(2008,1,1)
pend=datetime.date(2020,12,31)   
nn=len(tsd.loc[pstart:pend])    
for f in range(len(FRQ)):
    for isp in range(oosp*12,oosp*200,oosp*12):#10,pt):
        ps=pstart-relativedelta(months=isp*interval-1)
        tsdd=tsd.loc[ps:pend]
        if (len(tsdd)-isp==nn) or (len(tsdd)-isp==nn-1):
            real0,real9,pred=backtest_VECM(tsdd,isp,oosp,'ci')
            performance_report(real0,real9,pred)

VECM                   12 mse_r  1.6319 mse_p    0.03 wr  0.5532 wrp    0.23 len   47
VECM                   24 mse_r  1.1537 mse_p    0.06 wr  0.5319 wrp    0.33 len   47
VECM                   36 mse_r  1.0371 mse_p    0.06 wr  0.5745 wrp    0.15 len   47
VECM                   48 mse_r  1.0102 mse_p    0.26 wr  0.5106 wrp    0.44 len   47
VECM                   60 mse_r  1.0229 mse_p    0.07 wr  0.5319 wrp    0.33 len   47
VECM                   72 mse_r  1.0020 mse_p    0.41 wr  0.5319 wrp    0.33 len   47
VECM                   84 mse_r  1.0311 mse_p    0.02 wr  0.4681 wrp    0.33 len   47


In [18]:
model='VECM'
oosp=4
pstart=datetime.date(1991,1,1)
pend=datetime.date(1995,12,31)   
nn=len(tsd.loc[pstart:pend])    
for f in range(len(FRQ)):
    for isp in range(oosp*2,oosp*200,oosp*1):#10,pt):
        ps=pstart-relativedelta(months=isp*interval-1)
        tsdd=tsd.loc[ps:pend]
        if (len(tsdd)-isp==nn) or (len(tsdd)-isp==nn-1):
            real0,real9,pred=backtest_VECM(tsdd,isp,oosp,'ci')
            performance_report(real0,real9,pred)

VECM                    8 mse_r  1.0620 mse_p    0.32 wr  0.6250 wrp    0.16 len   16
VECM                   12 mse_r  0.7585 mse_p    0.07 wr  0.6250 wrp    0.16 len   16
VECM                   16 mse_r  0.7116 mse_p    0.04 wr  0.7500 wrp    0.02 len   16
VECM                   20 mse_r  0.6601 mse_p    0.02 wr  0.6250 wrp    0.16 len   16


In [19]:
model='VECM'
oosp=4
pstart=datetime.date(1995,1,1)
pend=datetime.date(2007,12,31)   
nn=len(tsd.loc[pstart:pend])    
for f in range(len(FRQ)):
    for isp in range(oosp*2,oosp*200,oosp*1):#10,pt):
        ps=pstart-relativedelta(months=isp*interval-1)
        tsdd=tsd.loc[ps:pend]
        if (len(tsdd)-isp==nn) or (len(tsdd)-isp==nn-1):
            real0,real9,pred=backtest_VECM(tsdd,isp,oosp,'ci')
            performance_report(real0,real9,pred)

VECM                    8 mse_r 144.4627 mse_p    0.15 wr  0.4375 wrp    0.19 len   48
VECM                   12 mse_r  1.8706 mse_p    0.02 wr  0.5208 wrp    0.39 len   48
VECM                   16 mse_r  1.3348 mse_p    0.05 wr  0.4583 wrp    0.28 len   48
VECM                   20 mse_r  1.0946 mse_p    0.22 wr  0.5833 wrp    0.12 len   48
VECM                   24 mse_r  1.2601 mse_p    0.02 wr  0.6250 wrp    0.04 len   48
VECM                   28 mse_r  1.4493 mse_p    0.00 wr  0.3542 wrp    0.02 len   48
VECM                   32 mse_r  1.6787 mse_p    0.00 wr  0.2708 wrp    0.00 len   48
VECM                   36 mse_r  1.2562 mse_p    0.01 wr  0.3333 wrp    0.01 len   48


In [20]:
model='VECM'
oosp=4
pstart=datetime.date(2008,1,1)
pend=datetime.date(2020,12,31)   
nn=len(tsd.loc[pstart:pend])    
for f in range(len(FRQ)):
    for isp in range(oosp*6,oosp*200,oosp*2):#10,pt):
        ps=pstart-relativedelta(months=isp*interval-1)
        tsdd=tsd.loc[ps:pend]
        if (len(tsdd)-isp==nn) or (len(tsdd)-isp==nn-1):
            real0,real9,pred=backtest_VECM(tsdd,isp,oosp,'ci')
            performance_report(real0,real9,pred)

VECM                   24 mse_r  1.4280 mse_p    0.00 wr  0.5227 wrp    0.38 len   44
VECM                   32 mse_r  1.2325 mse_p    0.00 wr  0.4091 wrp    0.11 len   44
VECM                   40 mse_r  1.1873 mse_p    0.02 wr  0.4545 wrp    0.27 len   44
VECM                   48 mse_r  1.0736 mse_p    0.02 wr  0.5000 wrp    0.50 len   44
VECM                   56 mse_r  1.0627 mse_p    0.03 wr  0.5455 wrp    0.27 len   44
VECM                   64 mse_r  1.0971 mse_p    0.00 wr  0.5000 wrp    0.50 len   44
VECM                   72 mse_r  1.0195 mse_p    0.13 wr  0.4318 wrp    0.18 len   44
VECM                   80 mse_r  1.0701 mse_p    0.00 wr  0.4773 wrp    0.38 len   44
VECM                   88 mse_r  1.0831 mse_p    0.01 wr  0.5455 wrp    0.27 len   44


In [21]:
model='VECM'
oosp=20
pstart=datetime.date(1995,1,1)
pend=datetime.date(2007,12,31)   
nn=len(tsd.loc[pstart:pend])    
for f in range(len(FRQ)):
    for isp in range(oosp*1,oosp*50,2):#10,pt):
        ps=pstart-relativedelta(months=isp*interval-1)
        tsdd=tsd.loc[ps:pend]
        if (len(tsdd)-isp==nn) or (len(tsdd)-isp==nn-1):
            real0,real9,pred=backtest_VECM(tsdd,isp,oosp,'ci')
            performance_report(real0,real9,pred)

VECM                   20 mse_r 14.4449 mse_p    0.01 wr  0.5625 wrp    0.24 len   32
VECM                   22 mse_r 10.9917 mse_p    0.00 wr  0.5000 wrp    0.50 len   32
VECM                   24 mse_r 11.7340 mse_p    0.00 wr  0.5000 wrp    0.50 len   32
VECM                   26 mse_r 13.7312 mse_p    0.00 wr  0.4062 wrp    0.14 len   32
VECM                   28 mse_r 16.7645 mse_p    0.00 wr  0.3438 wrp    0.04 len   32
VECM                   30 mse_r 22.4574 mse_p    0.00 wr  0.3125 wrp    0.02 len   32
VECM                   32 mse_r 34.9433 mse_p    0.02 wr  0.2188 wrp    0.00 len   32
VECM                   34 mse_r 19.7518 mse_p    0.00 wr  0.3125 wrp    0.02 len   32
VECM                   36 mse_r 12.8241 mse_p    0.01 wr  0.3438 wrp    0.04 len   32


In [22]:
model='VECM'
oosp=20
pstart=datetime.date(2008,1,1)
pend=datetime.date(2020,12,31)   
nn=len(tsd.loc[pstart:pend])    
for f in range(len(FRQ)):
    for isp in range(oosp*2,oosp*60,6):#10,pt):
        ps=pstart-relativedelta(months=isp*interval-1)
        tsdd=tsd.loc[ps:pend]
        if (len(tsdd)-isp==nn) or (len(tsdd)-isp==nn-1):
            real0,real9,pred=backtest_VECM(tsdd,isp,oosp,'ci')
            performance_report(real0,real9,pred)

VECM                   40 mse_r  1.9839 mse_p    0.00 wr  0.2500 wrp    0.00 len   28
VECM                   46 mse_r  1.5375 mse_p    0.00 wr  0.2500 wrp    0.00 len   28
VECM                   52 mse_r  1.3982 mse_p    0.00 wr  0.3214 wrp    0.03 len   28
VECM                   58 mse_r  1.3736 mse_p    0.00 wr  0.2500 wrp    0.00 len   28
VECM                   64 mse_r  1.2358 mse_p    0.00 wr  0.2857 wrp    0.01 len   28
VECM                   70 mse_r  1.1543 mse_p    0.00 wr  0.3214 wrp    0.03 len   28
VECM                   76 mse_r  1.2856 mse_p    0.00 wr  0.3571 wrp    0.07 len   28
VECM                   82 mse_r  1.4367 mse_p    0.00 wr  0.2500 wrp    0.00 len   28
VECM                   88 mse_r  1.4701 mse_p    0.00 wr  0.1786 wrp    0.00 len   28
